In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
import datetime
warnings.filterwarnings('ignore')

## Cardiogenic Shock

In [2]:
#IMPORT FILES NEEDED
shockOrig = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Cardiogenic Shock Full Data Set.csv", sep=",", index_col='PAT_ID').sort_index()
cath = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Cath Data.csv", sep=",", index_col='PAT_ID').sort_index()
echo = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Echo Data.csv", sep=",", index_col='PAT_ID').sort_index()


In [3]:
#For initial training, just use first row for each patient
shock = pd.DataFrame(columns=shockOrig.columns)
print("Num indexes = ", len(set(shockOrig.index)))

for idx in set(shockOrig.index):
    pt = shockOrig.loc[idx]
    if type(pt) != pd.Series:
        row = pt.iloc[0]
        shock = shock.append(row)
    else:
        shock = shock.append(pt)

shock = shock.sort_index()

Num indexes =  401


In [4]:
#Fix multiple values in hemo
hemoCols = ['RA_MEAN','PA_SYS','PA_DIA','PA_MEAN','PCW_MEAN','FICK_HR','FICKCO','FICKCI','TDCO','TDCI','PA_SAT']

for col in hemoCols:
    for idx in shock.index:
        pt = str(shock.loc[idx][col])
#         print(pt, type(pt))
        if not pd.isna(pt) and ';' in pt:
#             print("FIXING")
            spl = pt.split(';')
#             print(spl)
            shock.at[idx, col] = float(spl[0])
        else:
            shock.at[idx, col] = float(pt)
            
#             print("fixed value is:", shock.loc[idx][col])
            
shock.head(10)

,PAT_MRN_ID,PAT_SEX,PAT_DOB,IndexAdmissionCSN,IndexAdmitDt,IndexDischDt,Deceased,H2H,SystolicHF,DiastolicHF,...,PA_DIA,PA_MEAN,PCW_MEAN,FICK_HR,FICKCO,FICKCI,TDCO,TDCI,PA_SAT,DeathDate
Z1002839,1228899,Male,1959-02-25,2.000000e+11,2017-08-07,2017-08-13,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-13
Z101592,1328191,Male,1938-07-03,2.000000e+11,2018-04-13,2018-04-19,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-08
Z101721,488816,Male,1957-10-13,2.000000e+11,2017-10-31,2017-11-11,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11
Z1017923,1417048,Male,1956-06-07,2.000000e+11,2018-11-20,2018-12-07,1.0,1.0,1.0,NaN,...,20,26,15,99,4.83,1.87,7.50,2.91,55.1,2020-07-30
Z1024230,1496206,Male,1955-11-21,2.000000e+11,2018-04-06,2018-04-19,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1024990,1172504,Male,1959-01-25,2.000000e+11,2018-01-29,2018-02-01,1.0,NaN,NaN,NaN,...,33,39,31,115,6.3,3,5.67,2.70,47.5,2018-02-01
Z104044,479349,Male,1952-09-18,2.000000e+11,2018-05-13,2018-05-18,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18
Z1078182,1465439,Male,1957-10-29,2.000000e+11,2019-04-23,2019-05-13,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-21
Z107856,523865,Female,1964-02-24,2.000000e+11,2018-09-14,2018-11-08,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-22
Z1099417,1285229,Female,1981-07-27,2.000000e+11,2019-05-09,2019-06-07,NaN,NaN,1.0,NaN,...,34,44,21,111,3.39,2.06,NaN,NaN,52.5,NaN


In [5]:
cohort = pd.DataFrame()
cohort['ID'] = shock.index
cohort = cohort.set_index('ID')

lst = []
for idx in shock.index: #age at first admission
    pt = shock.loc[idx]
    
    dob = datetime.datetime.strptime(pt['PAT_DOB'], '%Y-%m-%d')
    admDate = datetime.datetime.strptime(pt['IndexAdmitDt'], '%Y-%m-%d')
    
    age = admDate.year - dob.year - ((admDate.month, admDate.day) < (dob.month, dob.day))
    lst.append(age)

cohort['Age'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['PAT_SEX'] == 'Male': 
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#will have 3 rows --> adm1, adm2 and adm3
cohort['BMI_adm1'] = shock['ADM1_BMI'].tolist()
cohort['BMI_adm2'] = shock['ADM2_BMI'].tolist()
cohort['BMI_adm3'] = shock['ADM3_BMI'].tolist()

#convert ejf from ranges to values for 3 adms
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM1_DischargeEF']
    if not pd.isna(ef):
        val = re.search(r'\d+', ef).group()
    else:
        val = None
    lst.append(val)
        
cohort['EjF_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM2_DischargeEF']
    if not pd.isna(ef):
        val = re.search(r'\d+', ef).group()
    else:
        val = None
    lst.append(val)
        
cohort['EjF_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM3_DischargeEF']
    if not pd.isna(ef):
        val = re.search(r'\d+', ef).group()
    else:
        val = None
    lst.append(val)
        
cohort['EjF_adm3'] = lst


#Initial Hosp Days
cohort['InitialHospDays_adm1'] = shock['ADM1_LOS'].tolist()
cohort['InitialHospDays_adm2'] = shock['ADM2_LOS'].tolist()
cohort['InitialHospDays_adm3'] = shock['ADM3_LOS'].tolist()

#Total Hosp days
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    a1 = pt['ADM1_LOS']
    a2 = pt['ADM2_LOS']
    a3 = pt['ADM3_LOS']
    if pd.isna(pt['ADM1_LOS']):
        a1 = 0
    if pd.isna(pt['ADM2_LOS']):
        a2 = 0
    if pd.isna(pt['ADM3_LOS']):
        a3 = 0
    
    los = a1 + a2 + a3
    lst.append(los)

cohort['TotalHospDays'] = lst
    
cohort.head(15)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays
ID,,,,,,,,,,,,
Z1002839,58,1,NaN,NaN,NaN,15,None,None,6,NaN,NaN,6.0
Z101592,79,1,19.3,18.3,21.1,20,None,None,6,3.0,8.0,17.0
Z101721,60,1,NaN,29.0,31.8,20,None,40,11,2.0,9.0,22.0
Z1017923,62,1,34.5,31.6,NaN,20,45,None,17,9.0,NaN,26.0
Z1024230,62,1,25.9,NaN,NaN,35,None,None,13,4.0,NaN,17.0
Z1024990,59,1,26.2,NaN,NaN,15,None,None,3,NaN,NaN,3.0
Z104044,65,1,NaN,NaN,NaN,20,None,None,5,3.0,2.0,10.0
Z1078182,61,1,28.8,NaN,NaN,15,None,None,20,NaN,NaN,20.0
Z107856,54,2,36.4,NaN,NaN,65,None,None,55,3.0,2.0,60.0


In [6]:
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['COPD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['DM'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['CKD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['HeartTx'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

cohort.head(5)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays,COPD,DIAB,RENAL,HTRANS
ID,,,,,,,,,,,,,,,,
Z1002839,58,1,NaN,NaN,NaN,15,None,None,6,NaN,NaN,6.0,0,0,0,0
Z101592,79,1,19.3,18.3,21.1,20,None,None,6,3.0,8.0,17.0,1,0,0,0
Z101721,60,1,NaN,29.0,31.8,20,None,40,11,2.0,9.0,22.0,0,1,0,0
Z1017923,62,1,34.5,31.6,NaN,20,45,None,17,9.0,NaN,26.0,0,1,1,0
Z1024230,62,1,25.9,NaN,NaN,35,None,None,13,4.0,NaN,17.0,0,1,1,0


In [7]:
# Labs
cohort['BUN_adm1'] = shock['ADM1_Labs_BUN'].tolist()
cohort['BUN_adm2'] = shock['ADM2_Labs_BUN'].tolist()
cohort['BUN_adm3'] = shock['ADM3_Labs_BUN'].tolist()

cohort['CRT_adm1'] = shock['ADM1_Labs_Cr'].tolist()
cohort['CRT_adm2'] = shock['ADM2_Labs_Cr'].tolist()
cohort['CRT_adm3'] = shock['ADM3_Labs_Cr'].tolist()

cohort['SOD_adm1'] = shock['ADM1_Labs_NA'].tolist()
cohort['SOD_adm2'] = shock['ADM2_Labs_NA'].tolist()
cohort['SOD_adm3'] = shock['ADM3_Labs_NA'].tolist()

cohort['WBC_adm1'] = shock['ADM1_Labs_Lymphocytes'].tolist()
cohort['WBC_adm2'] = shock['ADM2_Labs_Lymphocytes'].tolist()
cohort['WBC_adm3'] = shock['ADM3_Labs_Lymphocytes'].tolist()

#Meds
#ACE
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM1_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM1_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM2_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM2_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM3_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM3_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm3'] = lst

#BET
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm3'] = lst

#DIUR
cohort['DIUR_adm1'] = shock['ADM1_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm2'] = shock['ADM2_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm3'] = shock['ADM3_Diuretics__Potassium Sparing Diuretics'].tolist()

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,WBC_adm3,ACE_adm1,ACE_adm2,ACE_adm3,BET_adm1,BET_adm2,BET_adm3,DIUR_adm1,DIUR_adm2,DIUR_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,58,1,NaN,NaN,NaN,15,None,None,6,NaN,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
Z101592,79,1,19.3,18.3,21.1,20,None,None,6,3.0,...,0.93,0,0,1,0,0,0,NaN,NaN,1.0
Z101721,60,1,NaN,29.0,31.8,20,None,40,11,2.0,...,NaN,0,1,1,0,0,1,NaN,NaN,1.0
Z1017923,62,1,34.5,31.6,NaN,20,45,None,17,9.0,...,NaN,0,0,0,0,1,0,NaN,NaN,NaN
Z1024230,62,1,25.9,NaN,NaN,35,None,None,13,4.0,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z95784,83,2,34.7,33.3,NaN,50,55,None,26,4.0,...,NaN,0,1,0,0,0,0,NaN,NaN,NaN
Z96296,70,2,27.6,NaN,NaN,65,55,60,9,8.0,...,0.82,1,1,1,0,0,0,NaN,NaN,NaN
Z964794,61,1,NaN,NaN,28.0,None,None,15,15,5.0,...,2.11,0,0,1,0,0,1,1.0,1.0,NaN


In [8]:
#Invasive Hemo
# cohort['HRTRT_adm1'] = shock['ADM1_EKG_QRSD'].tolist()
# cohort['HRTRT_adm2'] = shock['ADM2_EKG_QRSD'].tolist()
# cohort['HRTRT_adm3'] = shock['ADM3_EKG_QRSD'].tolist()

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM1_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM2_EKG_QRSD']
    print(pt)
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM3_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm3'] = lst

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM1_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm1'] = sys
cohort['BPDIAS_adm1'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM2_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm2'] = sys
cohort['BPDIAS_adm2'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM3_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm3'] = sys
cohort['BPDIAS_adm3'] = dias

#HEMO
rap1 = []
pas1 = []
pad1 = []
pamn1 = []
pcwp1 = []
co1 = []
ci1 = []

rap2 = []
pas2 = []
pad2 = []
pamn2 = []
pcwp2 = []
co2 = []
ci2 = []

rap3 = []
pas3 = []
pad3 = []
pamn3 = []
pcwp3 = []
co3 = []
ci3 = []

for idx in shock.index: 
    pt = shock.loc[idx]
    adm1Date = None
    adm2Date = None
    adm3Date = None
    adms = []
    
    if not pd.isna(pt['CATH_DATE']):
        cathDate = datetime.datetime.strptime(pt['CATH_DATE'], '%Y-%m-%d')
        if not pd.isna(pt['ADM1_AdmitDt']):
            adm1Date = datetime.datetime.strptime(pt['ADM1_AdmitDt'], '%Y-%m-%d')
            adms.append(adm1Date)
        if not pd.isna(pt['ADM2_AdmitDt']):
            adm2Date = datetime.datetime.strptime(pt['ADM2_AdmitDt'], '%Y-%m-%d')
            adms.append(adm2Date)
        if not pd.isna(pt['ADM3_AdmitDt']):
            adm3Date = datetime.datetime.strptime(pt['ADM3_AdmitDt'], '%Y-%m-%d')
            adms.append(adm3Date)
            
        closest = min(adms, key=lambda x: abs(x - cathDate))
        
        if closest == adm1Date:
            rap1.append(float(pt['RA_MEAN']))
            pas1.append(float(pt['PA_SYS']))
            pad1.append(float(pt['PA_DIA']))
            pamn1.append(float(pt['PA_MEAN']))
            pcwp1.append(float(pt['PCW_MEAN']))
            co1.append(float(pt['FICKCO']))
            ci1.append(float(pt['FICKCI']))
            
            rap2.append(np.nan)
            pas2.append(np.nan)
            pad2.append(np.nan)
            pamn2.append(np.nan)
            pcwp2.append(np.nan)
            co2.append(np.nan)
            ci2.append(np.nan)
            
            rap3.append(np.nan)
            pas3.append(np.nan)
            pad3.append(np.nan)
            pamn3.append(np.nan)
            pcwp3.append(np.nan)
            co3.append(np.nan)
            ci3.append(np.nan)
            
        elif closest == adm2Date:
            rap2.append(float(pt['RA_MEAN']))
            pas2.append(float(pt['PA_SYS']))
            pad2.append(float(pt['PA_DIA']))
            pamn2.append(float(pt['PA_MEAN']))
            pcwp2.append(float(pt['PCW_MEAN']))
            co2.append(float(pt['FICKCO']))
            ci2.append(float(pt['FICKCI']))
            
            rap1.append(np.nan)
            pas1.append(np.nan)
            pad1.append(np.nan)
            pamn1.append(np.nan)
            pcwp1.append(np.nan)
            co1.append(np.nan)
            ci1.append(np.nan)
            
            rap3.append(np.nan)
            pas3.append(np.nan)
            pad3.append(np.nan)
            pamn3.append(np.nan)
            pcwp3.append(np.nan)
            co3.append(np.nan)
            ci3.append(np.nan)
            
        
        else: #closest == adm3Date
            rap3.append(float(pt['RA_MEAN']))
            pas3.append(float(pt['PA_SYS']))
            pad3.append(float(pt['PA_DIA']))
            pamn3.append(float(pt['PA_MEAN']))
            pcwp3.append(float(pt['PCW_MEAN']))
            co3.append(float(pt['FICKCO']))
            ci3.append(float(pt['FICKCI']))
            
            rap1.append(np.nan)
            pas1.append(np.nan)
            pad1.append(np.nan)
            pamn1.append(np.nan)
            pcwp1.append(np.nan)
            co1.append(np.nan)
            ci1.append(np.nan)
            
            rap2.append(np.nan)
            pas2.append(np.nan)
            pad2.append(np.nan)
            pamn2.append(np.nan)
            pcwp2.append(np.nan)
            co2.append(np.nan)
            ci2.append(np.nan)
            
            
        
    else:
        rap1.append(np.nan)
        pas1.append(np.nan)
        pad1.append(np.nan)
        pamn1.append(np.nan)
        pcwp1.append(np.nan)
        co1.append(np.nan)
        ci1.append(np.nan)

        rap2.append(np.nan)
        pas2.append(np.nan)
        pad2.append(np.nan)
        pamn2.append(np.nan)
        pcwp2.append(np.nan)
        co2.append(np.nan)
        ci2.append(np.nan)
        
        rap3.append(np.nan)
        pas3.append(np.nan)
        pad3.append(np.nan)
        pamn3.append(np.nan)
        pcwp3.append(np.nan)
        co3.append(np.nan)
        ci3.append(np.nan)

cohort['RAP_adm1'] = rap1
cohort['PAS_adm1'] = pas1
cohort['PAD_adm1'] = pad1
cohort['PAMN_adm1'] = pamn1
cohort['PCWP_adm1'] = pcwp1
cohort['CO_adm1'] = co1
cohort['CI_adm1'] = ci1

cohort['RAP_adm2'] = rap2
cohort['PAS_adm2'] = pas2
cohort['PAD_adm2'] = pad2
cohort['PAMN_adm2'] = pamn2
cohort['PCWP_adm2'] = pcwp2
cohort['CO_adm2'] = co2
cohort['CI_adm2'] = ci2

cohort['RAP_adm3'] = rap3
cohort['PAS_adm3'] = pas3
cohort['PAD_adm3'] = pad3
cohort['PAMN_adm3'] = pamn3
cohort['PCWP_adm3'] = pcwp3
cohort['CO_adm3'] = co3
cohort['CI_adm3'] = ci3



cohort.head(5)

nan
120.0
101.0
nan
nan
nan
174.0
nan
nan
nan
66.0
nan
108.0
nan
nan
nan
nan
nan
86.0
nan
82.0
115.0
nan
92.0
nan
197.0
140.0
nan
nan
170.0
158.0
nan
nan
89.0
110.0
nan
nan
186.0
85.0
101.0
120.0
211.0
nan
nan
162.0
120.0
nan
nan
nan
nan
224.0
82.0
116.0
107.0
78.0
146.0
107.0
186.0
nan
206.0
180.0
112.0
92.0
161.0
82.0
114.0
74.0
80.0
132.0
114.0
177.0
nan
112.0
nan
159.0
127.0
nan
nan
93.0
74.0
nan
nan
156.0
160.0
nan
102.0
nan
nan
nan
102.0
nan
nan
nan
nan
83.0
nan
126.0
nan
104.0
nan
52.0
107.0
nan
nan
148.0
nan
121.0
88.0
156.0
88.0
126.0
76.0
nan
120.0
99.0
146.0
102.0
85.0
166.0
68.0
nan
nan
nan
133.0
74.0
nan
nan
nan
175.0
140.0
199.0
174.0
nan
nan
nan
106.0
nan
nan
nan
74.0
84.0
nan
90.0
nan
nan
138.0
94.0
nan
69.0
nan
nan
nan
107.0
nan
nan
nan
nan
nan
nan
nan
163.0
nan
nan
232.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
212.0
nan
nan
nan
nan
nan
104.0
nan
66.0
nan
nan
nan
nan
nan
57.0
nan
nan
nan
102.0
nan
108.0
nan
nan
nan
152.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
100.

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,PCWP_adm2,CO_adm2,CI_adm2,RAP_adm3,PAS_adm3,PAD_adm3,PAMN_adm3,PCWP_adm3,CO_adm3,CI_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,58,1,NaN,NaN,NaN,15,None,None,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101592,79,1,19.3,18.3,21.1,20,None,None,6,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101721,60,1,NaN,29.0,31.8,20,None,40,11,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1017923,62,1,34.5,31.6,NaN,20,45,None,17,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1024230,62,1,25.9,NaN,NaN,35,None,None,13,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Composite Hemo

idx = sorted(set(cohort.index))

#MAP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]
    
    admit1 = t['BPSYS_adm1'] + ((2 * t['BPDIAS_adm1']) / 3)
    admit2 = t['BPSYS_adm2'] + ((2 * t['BPDIAS_adm2']) / 3)
    admit3 = t['BPSYS_adm3'] + ((2 * t['BPDIAS_adm3']) / 3)

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['MAP_adm1'] = a1
cohort['MAP_adm2'] = a2
cohort['MAP_adm3'] = a3

#MPAP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1= t['PAS_adm1'] + ((2 * t['PAD_adm1']) / 3)
    admit2= t['PAS_adm2'] + ((2 * t['PAD_adm2']) / 3)
    admit3= t['PAS_adm3'] + ((2 * t['PAD_adm3']) / 3)

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['MPAP_adm1'] = a1
cohort['MPAP_adm2'] = a2
cohort['MPAP_adm3'] = a3

#CPI
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = (t['CI_adm1'] * t['MAP_adm1']) / 451
    admit2 = (t['CI_adm2'] * t['MAP_adm2']) / 451
    admit3 = (t['CI_adm3'] * t['MAP_adm3']) / 451

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['CPI_adm1'] = a1
cohort['CPI_adm2'] = a2
cohort['CPI_adm3'] = a3
    
#PP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['BPSYS_adm1'] - t['BPDIAS_adm1']
    admit2 = t['BPSYS_adm2'] - t['BPDIAS_adm2']
    admit3 = t['BPSYS_adm3'] - t['BPDIAS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PP_adm1'] = a1
cohort['PP_adm2'] = a2
cohort['PP_adm3'] = a3

#PPP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['BPSYS_adm1']
    admit2 = t['PP_adm2'] / t['BPSYS_adm2']
    admit3 = t['PP_adm3'] / t['BPSYS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPP_adm1'] = a1
cohort['PPP_adm2'] = a2
cohort['PPP_adm3'] = a3

#PAPP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = (t['PAS_adm1']  - t['PAD_adm1']) / t['PAS_adm1']
    admit2 = (t['PAS_adm2']  - t['PAD_adm2']) / t['PAS_adm2']
    admit3 = (t['PAS_adm3']  - t['PAD_adm3']) / t['PAS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PAPP_adm1'] = a1
cohort['PAPP_adm2'] = a2
cohort['PAPP_adm3'] = a3


#SVR
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = 80 * (t['MAP_adm1']  - t['RAP_adm1']) / t['CO_adm1']
    admit2 = 80 * (t['MAP_adm2']  - t['RAP_adm2']) / t['CO_adm2']
    admit3 = 80 * (t['MAP_adm3']  - t['RAP_adm3']) / t['CO_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['SVR_adm1'] = a1
cohort['SVR_adm2'] = a2
cohort['SVR_adm3'] = a3

#RAT
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]
    admit1 = t['RAP_adm1'] / t['PCWP_adm1']
    admit2 = t['RAP_adm2'] / t['PCWP_adm2']
    admit3 = t['RAP_adm3'] / t['PCWP_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)

cohort['RAT_adm1'] = a1
cohort['RAT_adm2'] = a2
cohort['RAT_adm3'] = a3


#PPRatio
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['HRTRT_adm1']
    admit2 = t['PP_adm2'] / t['HRTRT_adm2']
    admit3 = t['PP_adm3'] / t['HRTRT_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPRatio_adm1'] = a1
cohort['PPRatio_adm2'] = a2
cohort['PPRatio_adm3'] = a3

#PAPi
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = (t['PAS_adm1'] - t['PAD_adm1']) / t['RAP_adm1']
    admit2 = (t['PAS_adm2'] - t['PAD_adm2']) / t['RAP_adm2']
    admit3 = (t['PAS_adm3'] - t['PAD_adm3']) / t['RAP_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PAPi_adm1'] = a1
cohort['PAPi_adm2'] = a2
cohort['PAPi_adm3'] = a3


#SAPi
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = (t['BPSYS_adm1'] - t['BPDIAS_adm1']) / t['PCWP_adm1']
    admit2 = (t['BPSYS_adm2'] - t['BPDIAS_adm2']) / t['PCWP_adm2']
    admit3 = (t['BPSYS_adm3'] - t['BPDIAS_adm3']) / t['PCWP_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['SAPi_adm1'] = a1
cohort['SAPi_adm2'] = a2
cohort['SAPi_adm3'] = a3

#CPP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['BPDIAS_adm1'] - t['PCWP_adm1']
    admit2 = t['BPDIAS_adm2'] - t['PCWP_adm2']
    admit3 = t['BPDIAS_adm3'] - t['PCWP_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['CPP_adm1'] = a1
cohort['CPP_adm2'] = a2
cohort['CPP_adm3'] = a3

#PRAPRat
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['RAP_adm1']
    admit2 = t['PP_adm2'] / t['RAP_adm2']
    admit3 = t['PP_adm3'] / t['RAP_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PRAPRat_adm1'] = a1
cohort['PRAPRat_adm2'] = a2
cohort['PRAPRat_adm3'] = a3


cohort.head(5)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,PAPi_adm3,SAPi_adm1,SAPi_adm2,SAPi_adm3,CPP_adm1,CPP_adm2,CPP_adm3,PRAPRat_adm1,PRAPRat_adm2,PRAPRat_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,58,1,NaN,NaN,NaN,15,None,None,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101592,79,1,19.3,18.3,21.1,20,None,None,6,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101721,60,1,NaN,29.0,31.8,20,None,40,11,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1017923,62,1,34.5,31.6,NaN,20,45,None,17,9.0,...,NaN,2.066667,NaN,NaN,56.0,NaN,NaN,3.444444,NaN,NaN
Z1024230,62,1,25.9,NaN,NaN,35,None,None,13,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Make separate datasets

#HEMO
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row2 = []
    row0.append(df.name)
    row1.append(df.name)
    row2.append(df.name)

    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
            row2.append(df[col])
        elif col + "_adm1" in cohort.columns and col + "_adm2" in cohort.columns and col + "_adm3" in cohort.columns:
            row0.append(df[col+"_adm1"])
            row1.append(df[col+"_adm2"])
            row2.append(df[col+"_adm3"])
#         elif col + "_adm1" in cohort.columns and not col + "_adm2" in cohort.columns:
#             row0.append(df[col+"_B"])
#             row1.append(0)
#         elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
#             row0.append(0)
#             row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
            row2.append(0)
    
    giantList.append(row0)
    giantList.append(row1)
    giantList.append(row2)

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
hemoDF

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,58,1,0,15,NaN,NaN,NaN,NaN,NaN,NaN,...,42.0,0.461538,NaN,NaN,NaN,0.451613,NaN,NaN,NaN,NaN
Z1002839,58,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1002839,58,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101592,79,1,0,20,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,0.460000,NaN,NaN,NaN,0.380165,NaN,NaN,NaN,NaN
Z101592,79,1,0,None,NaN,NaN,NaN,NaN,NaN,NaN,...,38.0,0.422222,NaN,NaN,NaN,0.316667,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z977261,47,2,0,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z977261,47,2,0,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z988224,19,1,0,70,8.0,20.0,12.0,15.0,8.0,NaN,...,42.0,0.350000,0.4,NaN,1.0,0.355932,1.0,5.25,70.0,5.25


In [11]:
#Get only patients with hemo
hemoTest = hemoDF[['RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 'CI']]
hemoComposite = hemoDF.loc[~(np.isnan(hemoTest)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)


(152, 28)


,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
Z1017923,62,1,0,20,9.0,38.0,20.0,26.0,15.0,4.83,...,31.0,0.303922,0.473684,2324.361629,0.600000,0.276786,2.000000,2.066667,56.0,3.444444
Z1024990,59,1,0,15,12.0,51.0,33.0,39.0,31.0,6.30,...,70.0,0.555556,0.352941,1921.693122,0.387097,0.752688,1.500000,2.258065,25.0,5.833333
Z1099417,37,2,0,25,32.0,58.0,34.0,44.0,21.0,3.39,...,41.0,0.445652,0.413793,2218.289086,1.523810,0.488095,0.750000,1.952381,30.0,1.281250
Z1140320,61,2,0,25,19.0,54.0,24.0,38.0,36.0,4.20,...,37.0,0.352381,0.555556,2501.587302,0.527778,0.172093,1.578947,1.027778,32.0,1.947368
Z114276,70,1,0,50,4.0,24.0,8.0,15.0,7.0,4.62,...,36.0,0.367347,0.666667,2343.434343,0.571429,0.292683,4.000000,5.142857,55.0,9.000000


In [12]:
#save to file
hemoComposite.to_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/HemoCardiogenicShock.csv")

In [13]:
#MAke Hemo Labels
labels = pd.DataFrame()
labels['ID'] = hemoComposite.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in hemoComposite.index:
    pt = shock.loc[idx]
    if pt['Deceased'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

,Death
ID,
Z1017923,1
Z1024990,1
Z1099417,0
Z1140320,1
Z114276,1
...,...
Z927126,1
Z928524,1
Z93161,1


In [14]:
labels.to_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/HemoLabelsCardiogenicShock.csv")

In [15]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row2 = []
    row0.append(df.name)
    row1.append(df.name)
    row2.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
            row2.append(df[col])
        elif col + "_adm1" in cohort.columns and col + "_adm2" in cohort.columns and col + "_adm3" in cohort.columns:
            row0.append(df[col+"_adm1"])
            row1.append(df[col+"_adm2"])
            row2.append(df[col+"_adm3"])
#         elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
#             row0.append(df[col+"_B"])
#             row1.append(0)
#         elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
#             row0.append(0)
#             row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
            row2.append(0)
    
    giantList.append(row0)
    giantList.append(row1)
    giantList.append(row2)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,58,1,0,0,NaN,6.0,6.0,0,0,0,...,NaN,15,49.0,91.0,0,0,123.666667,42.0,0.461538,0.451613
Z1002839,58,1,0,0,NaN,NaN,6.0,0,0,0,...,NaN,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z1002839,58,1,0,0,NaN,NaN,6.0,0,0,0,...,NaN,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z101592,79,1,0,0,19.3,6.0,17.0,0,0,0,...,NaN,20,54.0,100.0,0,0,136.000000,46.0,0.460000,0.380165
Z101592,79,1,0,0,18.3,3.0,17.0,0,0,0,...,NaN,None,52.0,90.0,0,0,124.666667,38.0,0.422222,0.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z977261,47,2,0,0,NaN,NaN,7.0,0,0,0,...,NaN,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z977261,47,2,0,0,NaN,NaN,7.0,0,0,0,...,NaN,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z988224,19,1,0,0,23.0,39.0,48.0,0,0,0,...,NaN,70,78.0,120.0,0,0,172.000000,42.0,0.350000,0.355932


In [16]:
#save to file
allData.to_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/AllDataCardiogenicShock.csv")

In [17]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = shock.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['Deceased'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst


labels

,Death
ID,
Z1002839,1
Z101592,1
Z101721,1
Z1017923,1
Z1024230,0
...,...
Z95784,1
Z96296,1
Z964794,1


In [18]:
labels.to_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv")